In [ ]:
%%python3 -m venv ../myenv
%%source ../myenv/bin/activate

In [ ]:
%pip install -r ../requirements.txt

In [ ]:
from dotenv import load_dotenv
import pandas as pd
import os

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [ ]:
data_folder = "data"
dev_file_path = os.path.join(data_folder, "merged_instructions.csv")
df = pd.read_csv(dev_file_path)
df['english_instruction'].head()

In [ ]:
from openai import OpenAI

client = OpenAI()
result_data = []

for index, row in df.iterrows():
    message = row['english_instruction']

    user_message = "Write a python function that calculates the following: \n" \
                f"{message} \n" \
                "The method signature should be according to the given in the example part.\n" \
                "Use the following guidelines in generation: \n" \
                " - Provide the code only\n" \
                " - Do not output any other texts or test cases results.\n" \
                " - Include the necessary import statements on the top, even if they are usually not needed like - re, typing, itertools, Split etc. \n" \
                " - Do not include any comments in the code\n" \
                " - Do not forget to consider the edge cases\n" \
                " - Lastly, verify the code has no compilation errors and contains the necessary imports."

    response = client.responses.create(
        model="gpt-5",
        reasoning={"effort": "medium"},
        max_output_tokens=2048,
        input=[
            {
                "role": "user", 
                "content": user_message
            }
        ]
    )
    output = response.output_text
    print(output)

    result_data.append({
        "id": row['id'],
        "response": output
    })

    break

result_df = pd.DataFrame(result_data)

In [ ]:
# Create a json from the result_df
result_json = result_df.to_json(orient="records", indent=4)
# save json in the data_folder os.path.join(data_folder, "submission.json")
with open(os.path.join(data_folder, "submission-gpt5.json"), "w") as json_file:
    json_file.write(result_json)
